In [1]:
# Импортируем все необходимые библиотеки и задаем сид для рандомизатора
import pandas as pd
import numpy as np
import string
import pickle
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score, precision_recall_curve, confusion_matrix, accuracy_score, classification_report, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt_tab')

from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [31]:
# Загрузка данных
df = pd.read_csv('geo-reviews-dataset-2023.csv')
df_1 = df.loc[df['label'] == 1]
df_2 = df.loc[df['label'] == 2]
df_3 = df.loc[df['label'] == 3]
df_4 = df.loc[df['label'] == 4]
df_5 = df.loc[df['label'] == 5]

data = pd.concat([df_1[0:5000], df_2[0:5000], df_3[0:5000], df_4[0:5000], df_5[0:5000]], ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True) #перемешать
data.shape

(25000, 2)

In [32]:
data.head()

,review,label
0,"Данная студия закрывается, вторая через дорогу...",1
1,Какой может быть отзыв для единственного магаз...,4
2,"Были на поминках здесь, кухня неплохая, персон...",4
3,Хочу выразить благодарность персоналу в салоне...,5
4,"Самая худшая авиакомпания, которой приходилось...",1


In [33]:
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['label'], test_size = 0.25, random_state = 1)
y_train.value_counts()

label
1    3792
4    3769
2    3761
3    3735
5    3693
Name: count, dtype: int64

In [34]:
y_test.value_counts()

label
5    1307
3    1265
2    1239
4    1231
1    1208
Name: count, dtype: int64

In [35]:
#Предобработка текста
snowball = SnowballStemmer(language = "russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language = "russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [36]:
# Создаем словарь с наиболее часто встречаемыми словами
processed = data["review"]
processed = processed.apply(lambda x: " ".join(tokenize_sentence(x,  remove_stop_words = True)))
processed

0        дан студ закрыва втор дорог для нов клиент спл...
1        как отз единствен магазинчик район впихнут нев...
2        был поминк кухн неплох персона поддержива соот...
3        хоч выраз благодарн персонал салон `` дверн во...
4        сам худш авиакомпан котор приход лета 6 переле...
                               ...                        
24995    был класс интерактивн викторин очен интересн в...
24996    уютн семейн стамотолог цен умерен отношен тепл...
24997    не понрав пив 180р обычн светл закуск брал кол...
24998    отвратительн качеств окон работ нача июн 2022 ...
24999    ужасн мест всегд пыта навал нужн и текучк пост...
Name: review, Length: 25000, dtype: object

In [37]:
all_words = []
for text in processed:
    words = word_tokenize(text)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

# Print the result
print("Number of words: {}".format(len(all_words)))
print("Most common words: {}".format(all_words.most_common(15)))
word_features = [x[0] for x in all_words.most_common(2000)]

Number of words: 55678
Most common words: [('очен', 12394), ('эт', 11315), ('в', 7062), ('``', 6718), ('мест', 5537), ('так', 5371), ('хорош', 5365), ('котор', 5227), ('все', 5177), ('сам', 4795), ('номер', 4767), ('магазин', 4550), ('цен', 4476), ('прост', 4323), ('вкусн', 4215)]


In [38]:
# Функция для нахождения фич в тексте
def find_features(text):
    words = word_tokenize(text)
    features = {}
    for word in word_features:
        features[word] = word in words

    return features

In [39]:
# Обучение модели логистической регрессии
vectorizer = TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x,  remove_stop_words = True), token_pattern=None)
features = vectorizer.fit_transform(X_train)
logreg_model = LogisticRegression(random_state = 0)
logreg_model.fit(features, y_train)

d:\DataS\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [40]:
X = vectorizer.fit_transform(X_train)
y_pred = logreg_model.predict(X)

In [41]:
# Проверка правильности модели на конкретном примере
logreg_model.predict(features[40])

array([2], dtype=int64)

In [42]:
X_train.iloc[40]

'Унылый интерьер, очень темно, еда красиво подается, какие-то блюда вкусные, какие-то нет, но видно, что продукты, из которых готовятся блюда, в закупке максимально дешевые. Если уделить внимание качеству продуктов, то, думаю, всё будет вкусно. Обслуживание не понятное, подходят разные официанты, приносят блюда одному гостю, другие сидят ждут, или принесут горячее, потом холодное.'

In [43]:
logreg_model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)),
    ("model", LogisticRegression(random_state = 0 ))])
logreg_model_pipeline.fit(X_train, y_train)

d:\DataS\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x000001C581135EE0>)),
                ('model', LogisticRegression(random_state=0))])

In [44]:
# Получаем метрики точности
y_pred = logreg_model_pipeline.predict(X_test)
pd.DataFrame(
    confusion_matrix(y_test, y_pred)
)

,0,1,2,3,4
0,819,265,86,24,14
1,344,445,295,111,44
2,174,305,431,306,49
3,39,87,215,654,236
4,17,17,47,225,1001


In [45]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.536
Classification Report:
              precision    recall  f1-score   support

           1       0.59      0.68      0.63      1208
           2       0.40      0.36      0.38      1239
           3       0.40      0.34      0.37      1265
           4       0.50      0.53      0.51      1231
           5       0.74      0.77      0.76      1307

    accuracy                           0.54      6250
   macro avg       0.53      0.54      0.53      6250
weighted avg       0.53      0.54      0.53      6250



In [20]:
# Сохранение модели со словарем фич и функцие нахождения их в тексте
path = 'models\\naive_bayes_classifier.pickle'

with open(path, 'wb') as classifier_file:
    data_for_save = {
        'model': logreg_model,
        'features': word_features,
        'function': find_features,
        }
    pickle.dump(data_for_save, classifier_file)

In [21]:
#f = open('models\\naive_bayes_classifier.pickle', 'rb')
#sd = pickle.load(f)
#print(sd)

In [46]:
# Обучение байесовского классификатора
mulnb_model = MultinomialNB()
mulnb_model.fit(features, y_train)
mulnb_model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)),
    ("model", MultinomialNB())])

In [50]:
mulnb_model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x000001C59285FBA0>)),
                ('model', MultinomialNB())])

In [48]:
y_pred_B = mulnb_model_pipeline.predict(X_test)
pd.DataFrame(
    confusion_matrix(y_test, y_pred_B),
    #index=[["actual", "actual"], ["negative", "positive"]],
    #columns=[["predicted", "predicted"], ["negative", "positive"]],
)

,0,1,2,3,4
0,821,291,58,29,9
1,386,491,217,121,24
2,241,353,315,335,21
3,71,139,198,671,152
4,58,29,33,323,864


In [51]:
accuracy = accuracy_score(y_test, y_pred_B)
report = classification_report(y_test, y_pred_B)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.536
Classification Report:
              precision    recall  f1-score   support

           1       0.59      0.68      0.63      1208
           2       0.40      0.36      0.38      1239
           3       0.40      0.34      0.37      1265
           4       0.50      0.53      0.51      1231
           5       0.74      0.77      0.76      1307

    accuracy                           0.54      6250
   macro avg       0.53      0.54      0.53      6250
weighted avg       0.53      0.54      0.53      6250



# Дальше не смотреть

In [26]:
# Сохранение модели со словарем фич и функцие нахождения их в тексте
path = 'models\\logistic_regression_classifier.pickle'

with open(path, 'wb') as classifier_file:
    data_for_save = {
        'model': mulnb_model,
        'features': word_features,
        'function': find_features,
        }
    pickle.dump(data_for_save, classifier_file)

In [27]:
# Если мы хотим найти 95% негативных комментариев, то...
precision_score(y_test, y_pred)

0.8780487804878049

In [28]:
recall_score(y_test, y_pred)

0.9

In [29]:
prec, rec, thresholds = precision_recall_curve(y_test, probas_pred=mulnb_model_pipeline.predict_proba(X_test)[:, 1])

D:\DataS\Lib\site-packages\sklearn\metrics\_ranking.py:993: FutureWarning: probas_pred was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_score`` instead.
  warnings.warn(


In [30]:
np.where(prec > 0.95)

(array([47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
        64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75], dtype=int64),)

In [31]:
thresholds[53]

0.7046251291675192

In [32]:
pd.DataFrame(
    confusion_matrix(y_test, mulnb_model_pipeline.predict_proba(X_test)[:, 1] > thresholds[36]),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

predicted         
                 negative positive
actual negative        32        3
       positive         5       35

In [33]:
#Мы можем найти все 44 негативных комментария из 44 при thresholds[78]! thresholds[8] - Найдем все позитивные комментарии.
#При thresholds[36] наибольшая точность в 88%